In [4]:
%pip install langchain
%pip install langchain-google-genai
%pip install pinecone-client
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.
  Using cached langchain_google_genai-1.0.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_generativeai-0.4.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached protobuf-5.26.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.m

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/215.9 kB ? eta -:--:--
   ----- --------------------------------- 30.7/215.9 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 215.9/215.9 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/290.4 kB 660.6 kB/s eta 0:00:01
   --------------- ------------------------ 112.6/290.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 290.4/290.4 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
